<a href="https://colab.research.google.com/github/ctrivino1/YLearn/blob/main/Frisch_Waugh_Theorem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# data
from sklearn.datasets import fetch_california_housing
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import pandas as pd
#from ylearn.estimator_model.double_ml import DoubleML
from xgboost import XGBRegressor
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

housing = fetch_california_housing(as_frame=True)
data = housing.frame
outcome = housing.target_names[0]
data[outcome] = housing.target
data.head()

In [ ]:
import seaborn as sns
sns.regplot(x="MedInc", y="MedHouseVal", data=data, ci=False, line_kws={'color':'r', 'label':'linear fit'})
plt.legend()
plt.title(f"Sales and coupon usage");

Normal Regression model

In [ ]:
import statsmodels.formula.api as smf
smf.ols('MedHouseVal ~ MedInc	+ HouseAge +	AveRooms +	AveBedrms +	Population +	AveOccup +	Latitude +	Longitude	', data).fit().summary().tables[1]

Now lets prove the Frisch-Waugh-Lowell theorem actually holds

In [ ]:
# Create a model that predicts medhouseval based on medincs residuals

data['MedInc_tilde'] = smf.ols('MedInc ~ HouseAge +	AveRooms +	AveBedrms +	Population +	AveOccup +	Latitude +	Longitude', data).fit().resid

smf.ols('MedHouseVal ~ MedInc_tilde - 1', data).fit().summary().tables[1]

In [ ]:
# Create a model that predicts medhouseval based on medincs residuals

data['HouseAge_tilde'] = smf.ols('HouseAge ~  MedInc +	AveRooms +	AveBedrms +	Population +	AveOccup +	Latitude +	Longitude', data).fit().resid

smf.ols('MedHouseVal ~ HouseAge_tilde -1', data).fit().summary().tables[1]

In [ ]:
# create primary model residuals
data['MedHouseVal_tilde'] = smf.ols('MedHouseVal ~ AveRooms +	AveBedrms +	Population +	AveOccup +	Latitude +	Longitude', data).fit().resid
# create model that uses the residuals from the auxillary residuals to predict the primary models resdiduals
smf.ols('MedHouseVal_tilde ~ MedInc_tilde + HouseAge_tilde - 1', data).fit().summary().tables[1]

In [ ]:

data["MedInc_hat"] =  smf.ols('MedInc ~ HouseAge +	AveRooms +	AveBedrms +	Population +	AveOccup +	Latitude +	Longitude', data).fit().predict()

ax = sns.regplot(x="MedHouseVal", y="MedInc", data=data, ci=False, line_kws={'color':'r', 'label':'linear fit'})
ax.vlines(data["MedHouseVal"],
   np.minimum(data["MedInc"], data["MedInc_hat"]),
   np.maximum(data["MedInc"], data["MedInc_hat"]),
   linestyle='--', color='k', alpha=0.5, linewidth=1, label="residuals")
plt.legend()
plt.title(f"Coupons usage, income and residuals");

In [ ]:
sns.regplot(x="MedInc_tilde", y="MedHouseVal_tilde", data=data, ci=False, line_kws={'color':'r', 'label':'linear fit'})
plt.legend()
plt.title(f"Residual sales and residual coupons");

In [ ]:
data['MedHouseVal_tilde_scaled'] = data['MedHouseVal_tilde'] + np.mean(data['MedHouseVal'])
data['MedInc_tilde_scaled'] = data['MedInc_tilde'] + np.mean(data['MedInc'])

In [ ]:
sns.regplot(x="MedInc_tilde_scaled", y="MedHouseVal_tilde_scaled", data=data, ci=False, line_kws={'color':'r', 'label':'linear fit'})
plt.legend()
plt.title(f"Residual sales scaled and residual coupons scaled");